In [12]:
import numpy as np
from numpy.linalg import multi_dot
from numpy.linalg import qr
import numpy.linalg as npla
import scipy as sp
from scipy.linalg import block_diag, logm, eigvals
import pandas as pd
from multiprocessing import Pool
from functools import partial
import multiprocessing
from multiprocessing import set_start_method
from multiprocessing import get_context
import joblib
import time
import sympy

In [2]:
thetacrit = np.pi/4

In [3]:
'''
These functions match to Holden's tlInCore and TlOutCore functions
They construct the A and B type transfer matricies
'''

def TAMatrix(theta, phi_one,phi_two,phi_three):
    '''mat = np.array(
    [[np.exp(1j*phi_two)*(1/np.cos(theta)),  -np.exp(-1j *phi_one)*np.tan(theta)],
     [np.exp(1j*(phi_two-phi_three))*(np.tan(theta)), np.exp(-1j*phi_one - 1j*phi_three)*(1/np.cos(theta))]]
    )'''
    matrix_one = np.array([[1,0],[0,np.exp(-1j*phi_three)]])
    matrix_two = np.array([[1,-np.sin(theta)],[-np.sin(theta),1]])
    matrix_three = np.array([[np.exp(1j*phi_two),0],[0,np.exp(-1j*phi_one)]])
    
    return (1/np.cos(theta))*multi_dot([matrix_one,matrix_two,matrix_three])


def TBMatrix(theta, phi_one,phi_two,phi_three):   
    matrix_one = np.array([[np.exp(-1j*phi_three),0],[0,np.exp(1j*phi_one)]])
    matrix_two = np.array([[1,np.cos(theta)],[np.cos(theta),1]])
    matrix_three = np.array([[-1,0],[0,np.exp(1j*phi_two)]])
    
    return (1/np.sin(theta)*multi_dot([matrix_one,matrix_two,matrix_three]))

In [4]:
'''
These functions here named TAS and TBS construct strips of TA type or TB type matrices with set strip_width
'''

#enter interger value for x that corresponds to sample width
def TAS(theta,strip_width):
    Tslist = [ 
        TAMatrix(theta,*(2*np.pi)*np.random.random_sample(3)) for i in range(strip_width)
    ]
    return block_diag(*Tslist)

def TBS(theta,strip_width):
    Tslist = [ 
        TBMatrix(theta,*(2*np.pi)*np.random.random_sample(3)) for i in range(strip_width-1)
    ]
    #extra = TBMatrix((2*np.pi)*np.random.random_sample(),*(2*np.pi)*np.random.random_sample(3))
    extra = TBMatrix(theta,*(2*np.pi)*np.random.random_sample(3))
    temp_mat = block_diag(extra[1,1],*Tslist,extra[0,0])    
    temp_mat[0,(2*strip_width)-1] = extra[1,0]
    temp_mat[(2*strip_width)-1,0] = extra[0,1]
    return temp_mat

'''
Since each TA strip is followed by a TB strip, instead of alternating functions, its easier to combine them into a single strip.
The final strip length that you run later is thus twice as long as specified. 
'''
def FullStrip(theta,strip_width):
    return np.matmul(TAS(theta,strip_width),TBS(theta,strip_width))

In [16]:
'''#runtime can be increased by tossing the R matrix after determining sum of logs of diagonals 
'''
Full transfer does the QR decomposition and calcuates the transfer matrix for the entire sample. 
Note that strip_length, as mentioned in the last cell, is actually the half strip length. 
The value calculated is actually double the specified length.
'''

def FullTransfer(strip_length,strip_width,theta):  
    Tone = FullStrip(theta,strip_width)
    qone,rone = qr(Tone)
    bigQ = qone
    rlog_one = np.log(np.absolute(rone.diagonal()))
    for i in range(strip_length-1):
        matrixb = np.matmul(FullStrip(theta,strip_width),bigQ)
        q,r = qr(matrixb)
        bigQ = q
        rlogs = np.log(np.absolute(r.diagonal()))
        rlog_one = np.add(rlogs,rlog_one)
    return rlog_one'''

In [53]:
#Condition Number is |A^-1||A|
cond_num = []
for i in np.linspace(thetacrit,1.5,15):
    mat = FullStrip(i,32)
    cond_num.append(npla.norm(npla.inv(mat), ord = 'nuc')*npla.norm(mat, ord = 'nuc'))

In [54]:
cond_num

[18532.38004918444,
 18703.22812634886,
 19230.490061936045,
 20161.040200047843,
 21582.972883756287,
 23644.1812097785,
 26587.6968140671,
 30819.790268226694,
 37047.88092665319,
 46581.40149616522,
 62057.23195661999,
 89440.86175861907,
 144697.1519753519,
 283501.59690854663,
 824752.2942322484]

In [56]:
test= FullStrip(thetacrit,16)
p,q,r = lu(test)
npla.norm(npla.inv(q),ord = 'nuc')*npla.norm(q, ord = 'nuc')

NameError: name 'lu' is not defined